In [1]:
import torch
import sys
from swift.llm import get_model_tokenizer
dirs = ["/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
from custom import CustomModelType, CustomTemplateType
model_type = CustomModelType.openchat_35
model, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)


2024-04-02 18:52:20,348 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-02 18:52:20,351 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-04-02 18:52:20,387 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/openchat-3.5-0106
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/openchat-3.5-0106",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e

In [2]:
import jsonlines
from tqdm.notebook import tqdm

dict_list = []
version = "1"
with jsonlines.open(f"data{version}.jsonl", mode="r") as file_jsonl:
    for item in file_jsonl.iter():
        dict_list.append(item)

lens1 = []
lens2 = []

for item in tqdm(dict_list):
    lens1.append(len(tokenizer(item["query"])["input_ids"]))
    # lens2.append(len(tokenizer.tokenize(item["query"])))



  0%|          | 0/12192 [00:00<?, ?it/s]

In [3]:
# 3705

# [3675, 3674, 3671, 3665, 3662, 3662, 3640, 3638, 3636, 3632, 3616, 3604]


sorted(lens1, reverse=False)[:12]


[521, 724, 736, 750, 751, 758, 762, 771, 774, 776, 777, 778]

修改较长的句子

显卡 24GB

7B模型，长度控制在3600以下 


In [8]:
del_ids = []
for i, item in enumerate(lens1):
    if item == 521:
        del_ids.append(i)

del_ids


[248]

In [6]:
# print(len(tokenizer(dict_list[4157]["query"])["input_ids"]))
dict_list[614]["query"]


'According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-11-06\nContent: Former scientific advisor to Pfizer Michael Yeadon claims that the pandemic is fundamentally over in the U.K., and, we now know loads of people had prior immunity to COVID-19\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: 2020-11-28\nTitle: Science Feedback: Ex Pfizer Vice President, Michael Yeadon, made ...\nContent:\n"After drugmaker Pfizer’s former Vice President Michael Yeadon claimed that “the pandemic is fundamentally over in the UK,” health #fact_checker Health Feedback termed his assertions “#false”. KD #Covid_19 https://t.co/2YCTx30Hpq— PIDA STS 2020 (@PidaSts) November 27, 2020 ... \u2066@fordnation\u2069 Dr. Michael Yeadon, VP, Chief Scientist of Pfizer (ret) “no need for va

In [9]:
print(dict_list[248]["query"])


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-04-13
Content: Claim that the coronavirus brought something strange in the skies of town of Tetovo, North Macedonia, meaning a strange looking cloud. There is also a claim that the appearance of this cloud caused by the air, being much cleaner since lot of factories are shut down.

PRIOR KNOWLEDGE:
Information 1:
Publication date: 2020-04-13
Title: FALSE: Claim that the coronavirus brought something strange in ...
Content:
Explanation: There is nothing strange about this cloud. It is a lenticular cloud, and they existed before the COVID-19 pandemic. The experts say that while it is a rare appearance it has nothing to do with the coronavirus or with the cleaner air, but with the air movements happening in the trop